## **Experiment Analysis Data Analyst Challenge**

**Submitted by Kyle Ross - 13/01/2022**

**Phone**: 0428161103
**Email**: kylerossau@gmail.com
**Linkedin**: https://www.linkedin.com/in/kylerossau/

--------------------------------------------------

## **Part 2: Experiment Analysis**

Several functions have been written to easily run A/B Tests across segments of the data. The analysis itself can be found after the function definitions.

--------------------------------------------------

**Importing packages**:

In [1]:
import pandas as pd
import numpy as np
import math
import warnings
from scipy.stats import norm
from IPython.display import display, HTML

--------------------------------------------------

**Preparation**:

In [2]:
#Location of csv file
csv_target = r"C:\Users\kylec\OneDrive\Documents\Career\Data Challenges\Data Analyst\Provided Material\manual_elevations_experiment_data.csv"

#Importing the csv as a DataFrame
df_elevations = pd.read_csv(csv_target)

#Renaming the experiment groups as 'A - Auto and Manual' and 'B - Auto Only'
df_elevations['experiment_group'] = df_elevations['experiment_group'].apply(lambda x: 'A - Auto and Manual' if (x == 'A') else 'B - Auto Only')

#Setting a variable for the A/B grouping column
group_dimensions = r'experiment_group'

--------------------------------------------------

**subset_build Function to subset the dataframe by a specific column**:


experiment_group 'S' is excluded in all subsets as this is not used for A/B testing. This function is used to pass segmented subsets to other functions.

Takes two arguments:
 - **col_name** - the name of the column to be the basis of the subset
 - **filter_val** - the value in that column to filter the dataframe

In [3]:
def subset_build(col_name,filter_val):
    cond1 = df_elevations['experiment_group'] != 'S'
    cond2 = df_elevations[col_name] == filter_val
    df_subset_var = df_elevations[cond1 & cond2]
    return df_subset_var

--------------------------------------------------

**df_build_agg Function to build grouped dataframe by A/B Test, create aggregated fields, and generate rate metrics for use in the A/B testing function.**

Takes one argument:

 - **df_subset** - a dataframe representing the desired subset of data

In [4]:
def df_build_agg(df_subset):

    #Aggregations
    df_grp_agg = df_subset.groupby(group_dimensions).agg(count_searches=('search_id',np.size),
                                                         sum_elevations = ('num_elevations',np.sum), 
                                                         sum_all_clicks = ('num_clicks',np.sum),
                                                         sum_unique_clicks = ('num_clicks',np.count_nonzero),
                                                         sum_all_exports = ('num_exported_results',np.sum),
                                                         sum_unique_exports = ('num_exported_results',np.count_nonzero),
                                                         sum_all_licenses = ('num_licenses',np.sum),
                                                         sum_unique_licenses = ('num_licenses',np.count_nonzero))

    #Adding Derived Fields
    df_grp_agg['click_rate'] = (df_grp_agg['sum_all_clicks'] / df_grp_agg['count_searches'])
    df_grp_agg['unq_click_rate'] = (df_grp_agg['sum_unique_clicks'] / df_grp_agg['count_searches'])
    df_grp_agg['export_rate'] = (df_grp_agg['sum_all_exports'] / df_grp_agg['count_searches'])
    df_grp_agg['unq_export_rate'] = (df_grp_agg['sum_unique_exports'] / df_grp_agg['count_searches'])
    df_grp_agg['license_rate'] = (df_grp_agg['sum_all_licenses'] / df_grp_agg['count_searches'])
    df_grp_agg['unq_license_rate'] = (df_grp_agg['sum_unique_licenses'] / df_grp_agg['count_searches'])
    

    return df_grp_agg

--------------------------------------------------

**Function to run the A/B test and produce statistical metrics. Either prints results to console or returns a dictionary of results**

Takes 5 arguments:

 - **df_input**: takes the output of the df_build_agg function
 - **conversions_col_name**: the name of the column containing the conversion metric (i.e. clicks)
 - **rate_col_name**: the name of the column containing the conversion rate (i.e. clicks per search)
 - **print_title**: optional argument specifying the title of the printout
 - **return_dict**: optional argument to specify if a dictionary is to be the output

In [5]:
def run_ab_test(df_input,conversions_col_name,rate_col_name,print_title = 'Experiment Data',return_dict = False):

    warnings.filterwarnings("ignore", category=RuntimeWarning) 

    c_index = df_input.columns.get_loc(conversions_col_name)
    r_index = df_input.columns.get_loc(rate_col_name)
    
    variation = {'searches':df_input.iloc[1]['count_searches'],'conversions':df_input.iloc[1][c_index],'rate':df_input.iloc[1][r_index]}
    control = {'searches':df_input.iloc[0]['count_searches'],'conversions':df_input.iloc[0][c_index],'rate':df_input.iloc[0][r_index]}
    
    variation['std_err'] = math.sqrt((variation['rate'] * (1 - variation['rate'])/variation['searches']))
    control['std_err'] = math.sqrt((control['rate'] * (1 - control['rate'])/control['searches']))
    
    results = {}
    
    if control['rate'] < variation['rate']:
        results['z-score'] = -( (control['rate'] - variation['rate']) / math.sqrt( (control['std_err'] ** 2) + (variation['std_err'] ** 2) ) )
    else:
        results['z-score'] = (control['rate'] - variation['rate']) / math.sqrt( (control['std_err'] ** 2) + (variation['std_err'] ** 2) )
    
    results['confidence'] = norm.cdf(results['z-score'])         
    results['percentage_delta'] = -(control['rate'] - variation['rate'])
    results['percentage_var'] = results['percentage_delta'] / control['rate']

    #return results as a dictionary or print results in console
    if return_dict == False:
        print(print_title)
        print('         ')
        print(display(df_input.iloc[:,[0,c_index,r_index]]))
        print('         ')
        print('Standard Error - Control - A: ' + str("{:.2}".format(control['std_err'])))
        print('Standard Error - Variation - B: ' + str("{:.2}".format(variation['std_err'])))
        print('         ')
        print('Rate Delta: ' + str("{:.2}".format(results['percentage_delta'])))
        print('Percentage Var: ' + str("{:.2%}".format(results['percentage_var'])))
        print('Confidence Level: ' + str("{:.2%}".format(results['confidence'])))
    else:
        results_dict = results
        results_dict['a_am_searches'] = control['searches']
        results_dict['a_am_conv'] = control['conversions']
        results_dict['a_am_rate'] = control['rate']
        results_dict['a_am_stderr'] = control['std_err']
        results_dict['b_ao_searches'] = variation['searches']
        results_dict['b_ao_conv'] = variation['conversions']
        results_dict['b_ao_rate'] = variation['rate']
        results_dict['b_ao_stderr'] = variation['std_err']
        return results_dict

    warnings.resetwarnings()

--------------------------------------------------

## 1. Analysis of the overall impact of manual elevations

Using the functions created above, we can easily analyse the impact of manual elevations. 

We will review 3 key metrics:

 - **Unique Click Through Rate** = ( Unique Clicks per search / count of searches)
 - **Unique Export Rate** = ( Unique Exports per search / count of searches)
 - **License Rate** = (Licenses / count of searches)

Unique values were used for clicks and exports as worse search elevation results may actually increase the per search clicks. By making this value unique, the analysis looks at impact in terms of whether they did or didn't click/export/licence at least once per session.

Licensing was left as count all, as this is a positive occurrence regardless.

--------------------------------------------------

In [6]:
run_ab_test(df_build_agg(df_elevations[df_elevations['experiment_group'] != 'S'])
                         ,'sum_unique_clicks',
                         'unq_click_rate',
                         print_title = 'All data - Unique Click Rate')

All data - Unique Click Rate
         


,count_searches,sum_unique_clicks,unq_click_rate
experiment_group,,,
A - Auto and Manual,840702,453325,0.539222
B - Auto Only,847675,481952,0.568558


None
         
Standard Error - Control - A: 0.00054
Standard Error - Variation - B: 0.00054
         
Rate Delta: 0.029
Percentage Var: 5.44%
Confidence Level: 100.00%


**Unique Click Rate**: The fully automated manual elevations used in experiment group B improved the unique click rate by +5.44%. In other words, +5.44% more users clicked at least once in group B. This result is statistically significant.

--------------------------------------------------

In [7]:
run_ab_test(df_build_agg(df_elevations[df_elevations['experiment_group'] != 'S'])
                         ,'sum_unique_exports',
                         'unq_export_rate',
                         print_title = 'All data - Unique Export Rate')

All data - Unique Export Rate
         


,count_searches,sum_unique_exports,unq_export_rate
experiment_group,,,
A - Auto and Manual,840702,233902,0.278222
B - Auto Only,847675,235097,0.277343


None
         
Standard Error - Control - A: 0.00049
Standard Error - Variation - B: 0.00049
         
Rate Delta: -0.00088
Percentage Var: -0.32%
Confidence Level: 89.88%


**Unique Export Rate**: The fully automated manual elevations used in experiment group B had very little effect on the unique export rate. 

*Note: Looking separately at the non-unique export rate there was a statistically significant -2.48% decrease.*

--------------------------------------------------

In [8]:
run_ab_test(df_build_agg(df_elevations[df_elevations['experiment_group'] != 'S'])
                         ,'sum_all_licenses',
                         'license_rate',
                         print_title = 'All data - Licence Rate')

All data - Licence Rate
         


,count_searches,sum_all_licenses,license_rate
experiment_group,,,
A - Auto and Manual,840702,24687,0.029365
B - Auto Only,847675,28783,0.033955


None
         
Standard Error - Control - A: 0.00018
Standard Error - Variation - B: 0.0002
         
Rate Delta: 0.0046
Percentage Var: 15.63%
Confidence Level: 100.00%



**Licence Rate**: The fully automated manual elevations used in experiment group B had a very positive effect on the licence rate, resulting in +15.63% more license purchases than group A. This result is statistically significant.

--------------------------------------------------

## Overall Summary
The fully automated manual elevations used in experiment group B had much better unique click-rate (+5.44%) and licence-rate (+15.63%) performance than the combination of manual and automated elevations in group A.
The test had almost no effect on the unique export rate.

--------------------------------------------------

## 2. Suggestions for further analysis:

- **Segment the test by User Type, Media Type and Query** to determine if particular manual elevations should be maintained, or if the automated elevations should be set for all queries. Conduct this analysis for Click Rate, Export Rate and Licence Rate.
- Investigate experiment performance over time, and by day of week. This may reveal patterns of behaviour unique to the weekend period or workdays for example.
- Segment by user and join in additional user information data - to see if test effectiveness varies by things like user time subscribed, relative level of familiarity with the platform and more.
- Does the number of clicks correlate to the number of elevations?

---------------------------------

## 3. Unique Click-rate Test Segmentation by User Tiers, Media Type and Query:

--------------------

**The all_tests function below combines the A/B test results of other functions to create multiple subsets of a given column grouping, allowing for quick segmented analysis. Returns a dataframe.**

Takes 4 arguments:

  - **my_df** = the top level dataframe to get the subset categories from
  - **ss_colname** = the column from which to derive the subsets
  - **cv_col** = the column indicating conversions for the chosen metric
  - **rate_col** = the column indicating conversion rate for the chosen metric

In [9]:
def all_tests(my_df, ss_colname, cv_col, rate_col):
    combined_dict = { ss_colname : [], 'percentage_delta' : [],
                      'percentage_var' : [], 'z-score' : [],
                      'confidence' : [], 'a_am_searches' : [], 'a_am_conv' : [],
                      'a_am_rate' : [],'a_am_stderr' : [], 'b_ao_searches' : [],
                      'b_ao_conv' : [], 'b_ao_rate' : [], 'b_ao_stderr' : []}


    for ss_value in my_df[ss_colname].unique():
        try:
            ab_results = run_ab_test(df_input = df_build_agg(df_subset = subset_build(ss_colname,ss_value)),
                                     conversions_col_name = cv_col,
                                     rate_col_name = rate_col,
                                     return_dict = True)
            
            for key in ab_results:
                combined_dict[key].append(ab_results[key])

            combined_dict[ss_colname].append(ss_value)
        except IndexError:
            continue            
    
    output_df = pd.DataFrame(combined_dict).set_index(ss_colname)
    output_df['total_searches'] = (output_df['a_am_searches'] + output_df['b_ao_searches'])
    output_df['total_conversions'] = (output_df['a_am_conv'] + output_df['b_ao_conv'])
    return output_df

--------------------------

**Creating the User Tiers, Media Types and Queries segmented A/B test dataframes**

In [10]:
user_tiers_ab_df  = (all_tests(df_elevations,'user_tier','sum_unique_clicks','unq_click_rate'))
media_types_ab_df =(all_tests(df_elevations,'media_types','sum_unique_clicks','unq_click_rate')) 
queries_ab_df     =(all_tests(df_elevations,'query','sum_unique_clicks','unq_click_rate')) 

**Function to format tables below**

In [11]:
def ab_df_format(df):
    selectedcols_df = df.loc[ : , 
                        ['total_searches' , 
                        'total_conversions',
                        'a_am_rate','b_ao_rate',
                        'percentage_var',
                        'confidence' ] ].rename(columns={'total_searches': 'Total Searches', 
                                        'total_conversions': 'Total Unique Clicks',
                                        'a_am_rate': 'Group A Click Rate',
                                        'b_ao_rate': 'Group B Click Rate',
                                        'percentage_var': 'Percentage Var',
                                        'confidence': 'Confidence'})

    return selectedcols_df


------

## User Tiers

In [12]:
display(ab_df_format(user_tiers_ab_df))

,Total Searches,Total Unique Clicks,Group A Click Rate,Group B Click Rate,Percentage Var,Confidence
user_tier,,,,,,
free,1179952.0,651361.0,0.538460,0.565530,0.050272,1.0
paid,508425.0,283916.0,0.541002,0.575537,0.063835,1.0


Unique click rate improved for both the free and paid tiers when exposed to automated manual elevations.

The improvement was +5.03% for free, and +6.38% for paid.

Interestly, the paid tier saw a slightly better improvement than the free tier.

------

## Media Types

In [13]:
display(ab_df_format(media_types_ab_df))

,Total Searches,Total Unique Clicks,Group A Click Rate,Group B Click Rate,Percentage Var,Confidence
media_types,,,,,,
RV,1675970.0,929321.0,0.539614,0.569247,0.054914,1.000000
V,5648.0,2952.0,0.531941,0.510455,-0.040392,0.945345
R,6759.0,3004.0,0.447073,0.441941,-0.011479,0.664369


The default Raster and Vector 'RV' filter improved the most with automated elevations, with unique click rate increasing by +5.49%. This filter made up the bulk of all searches.

However, the Vector Only 'V' filter saw unique click rate decrease by -4.04% when automated elevations were applied.

Similarly the Raster Only 'R' filter saw a decline of -1.15%, although this result is not statistically significant. 

------

## Queries

**Code formatting and creating output tables for different views of the queries A/B tests.**

In [14]:
format_df = ab_df_format(queries_ab_df)

filter1 = format_df['Confidence'] >= 0.95
filter2 = format_df['Total Searches'] >= 1000
filter3 = format_df['Percentage Var'] < 0
filter4 = format_df['Percentage Var'] > 0

subset_df_best = format_df[filter1 & filter2 & filter4]
subset_df_worst = format_df[filter1 & filter2 & filter3]

sorted_df_best = subset_df_best.sort_values('Percentage Var',ascending=False)
sorted_df_worst = subset_df_worst.sort_values('Percentage Var',ascending=True)

pd.set_option('display.max_rows', 500)

**The top most improved queries with >95% confidence and > 1000 searches**

These queries would perform better with fully automated manual elevations. Edge cases with <5% improvement should be reviewed.

In [15]:
display(sorted_df_best)

,Total Searches,Total Unique Clicks,Group A Click Rate,Group B Click Rate,Percentage Var,Confidence
query,,,,,,
photo,5172.0,1256.0,0.057918,0.411765,6.109420,1.000000
table,3369.0,1194.0,0.198742,0.493536,1.483297,1.000000
photos,1126.0,267.0,0.143116,0.327526,1.288537,1.000000
technology,10852.0,4011.0,0.261661,0.475781,0.818309,1.000000
flag,3996.0,1794.0,0.326643,0.570644,0.746995,1.000000
happy,2463.0,850.0,0.272651,0.413061,0.514979,1.000000
mothers day,19258.0,1917.0,0.079971,0.119062,0.488822,1.000000
computer,31334.0,14689.0,0.378257,0.555750,0.469240,1.000000
person,2354.0,1097.0,0.376522,0.551495,0.464710,1.000000


**The bottom most declined queries with >95% confidence, and >1000 searches**

The automated manual elevations for these queries should be reviewed. Consider whether displaying manual elevations instead would be better for these queries.

In [16]:
display(sorted_df_worst)

,Total Searches,Total Unique Clicks,Group A Click Rate,Group B Click Rate,Percentage Var,Confidence
query,,,,,,
box,20628.0,9202.0,0.564652,0.295812,-0.476116,1.000000
free,18034.0,2776.0,0.179727,0.126984,-0.293459,1.000000
fashion,1947.0,648.0,0.359050,0.304487,-0.151965,0.994783
line,73020.0,45574.0,0.665982,0.579921,-0.129225,1.000000
dot,9691.0,5840.0,0.637333,0.568779,-0.107563,1.000000
sushi,1993.0,966.0,0.505515,0.459669,-0.090692,0.979412
planet,2523.0,1393.0,0.577138,0.527516,-0.085980,0.993954
numbers,10145.0,5945.0,0.606178,0.564955,-0.068005,0.999988
lines,26476.0,14335.0,0.559763,0.522902,-0.065851,1.000000


## 4. Final Recommendations:

--------------------

With a Unique Click Rate lift of + 5.44% and a Licence Rate lift of +15.63%, Automated elevations could be enabled for all searches to see an immediate performance improvement on the platform.

However, there are areas for review and potential improvement:
- The Vector Only filter saw a unique click rate decrease of -4.04% with automated elevations. This should be further investigated on a query by query basis, and search results for this filter should be reviewed. Ultimately, manual elevations may represent a better choice for for this filter type.
- Several queries were identified that performed worse with automated elevations. The search results for these should be reviewed in detail to see if the results can be improved. Manual elevations may be a better option for these queries.
- Top performing queries were identified. These could be reviewed for best practice learnings.

--------
**By Kyle Ross - 13/01/2021**